In [1]:
import sys
sys.path.append('../')

from cube import Cube
from mcts import MCTS
from astar import Astar
from bfs import BFS

import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

plt.rcParams['figure.figsize'] = [15, 8]

In [2]:
max_scramble_depth = 5
num_per_scramble_depth = 30
num_iterations_per_solve = 200_000

# MCTS or Astar
# algorithm = MCTS(c = 5, nu = 0.1, num_iter = num_iterations_per_solve)
# algorithm = Astar(lam = 1, num_iter = num_iterations_per_solve, network_type = 'resnet')
algorithm = BFS(num_iter = num_iterations_per_solve)

In [3]:
total_attempts = []
averages_of_succesful_attempts = []
stds_of_succesful_attempts = []

for scramble_depth in range(1, max_scramble_depth + 1):
    number_of_attempts = 0
    succesful_attempts_lengths = []
    for _ in tqdm(range(num_per_scramble_depth)):
        while True:
            cube = Cube()
            cube.scramble(scramble_depth)
            solution, steps_taken = algorithm.solve(cube, num_iterations_per_solve, return_steps_taken = True)
            number_of_attempts += 1
            if solution is not None:
                break
        succesful_attempts_lengths.append(steps_taken)
    print('Attempts: ', succesful_attempts_lengths)
    print('Mean: ', np.mean(succesful_attempts_lengths))
    print("Std: ", np.std(succesful_attempts_lengths))
    total_attempts.append(number_of_attempts)
    averages_of_succesful_attempts.append(np.mean(succesful_attempts_lengths))
    stds_of_succesful_attempts.append(np.std(succesful_attempts_lengths))

100%|██████████| 30/30 [00:00<00:00, 223.87it/s]


Attempts:  [3, 9, 11, 9, 5, 4, 4, 1, 5, 5, 12, 8, 4, 7, 2, 2, 1, 6, 9, 4, 5, 9, 4, 3, 5, 11, 3, 4, 10, 5]
Mean:  5.666666666666667
Std:  3.0477678535099897


100%|██████████| 30/30 [00:01<00:00, 23.94it/s]


Attempts:  [41, 26, 97, 100, 87, 77, 35, 61, 113, 61, 71, 16, 99, 103, 48, 58, 89, 64, 62, 76, 49, 103, 87, 82, 113, 123, 110, 32, 98, 26]
Mean:  73.56666666666666
Std:  29.413923384834074


100%|██████████| 30/30 [00:10<00:00,  2.87it/s]


Attempts:  [1189, 934, 600, 1006, 764, 232, 910, 153, 925, 542, 820, 921, 128, 311, 680, 457, 252, 275, 438, 798, 1057, 966, 916, 690, 233, 919, 719, 566, 193, 989]
Mean:  652.7666666666667
Std:  311.08794933623227


100%|██████████| 30/30 [01:50<00:00,  3.69s/it]


Attempts:  [10432, 9015, 10937, 5226, 5854, 3048, 7552, 1495, 2875, 4950, 3661, 7520, 6880, 9599, 11201, 8265, 1882, 10893, 7696, 4748, 8664, 8663, 6797, 3262, 10695, 4971, 6510, 10024, 3167, 10695]
Mean:  6905.9
Std:  2957.3069883482394


  7%|▋         | 2/30 [00:49<11:37, 24.92s/it]

In [ ]:
print('Total attempts:', total_attempts)
plt.bar(range(1, max_scramble_depth + 1), averages_of_succesful_attempts)
plt.errorbar(range(1, max_scramble_depth + 1), averages_of_succesful_attempts, yerr = stds_of_succesful_attempts, fmt = 'none', ecolor = 'red')